# Data Acquisition

## Imports

In [39]:
!pip install bs4 requests selenium

In [40]:
import os
import time
import json

# Web Requests
import requests
from urllib.parse import urlparse

# Web Scraping
from bs4 import BeautifulSoup
import pandas as pd

# Web Automation (Selenium)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import io

### What this code does

- **Defines datasets to download**: A list of NGER dataset IDs (`ID0075` … `ID0243`) is specified.
- **Builds the API endpoint**: Uses a base OData URL with a `select=*` query to fetch full records for each dataset ID.
- **Prepares an output folder**: Resolves the current file’s directory, creates `data/source 1/Data` if it doesn’t exist.
- **Downloads each dataset**:
  - Sends an HTTP GET to the dataset-specific URL.
  - On HTTP 200:
    - Parses the response as JSON.
    - Supports two shapes:
      - An object with a `value` array.
      - A top-level array.
    - Converts the records into a Pandas DataFrame.
    - Saves the data as a CSV named `NGER.IDXXXX.csv` in `data/source 1/Data`.
    - Prints a success message.
  - On non-200 or JSON/processing error:
    - Prints a failure message with status code or exception details.

### Key behaviors and safeguards

- **Flexible JSON handling**: Works whether the API returns `{ "value": [...] }` or a plain list.
- **Idempotent directory creation**: Uses `os.makedirs(..., exist_ok=True)` to avoid errors if the folder already exists.
- **Per-dataset error reporting**: Continues through the list even if one dataset fails, logging the issue.

In [41]:
dataset_ids = ["ID0075", "ID0076", "ID0077", "ID0078", "ID0079",
               "ID0080", "ID0081", "ID0082", "ID0083", "ID0243"]

base_url = "https://api.cer.gov.au/datahub-public/v1/api/ODataDataset/NGER/dataset/{}?select%3D%2A"
cwd = os.getcwd()
data_folder = os.path.join(cwd, "data")
os.makedirs(data_folder, exist_ok=True)

for dataset_id in dataset_ids:
    url = base_url.format(dataset_id)
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()  # Parse JSON
            
            # Handle both dict with "value" key or list directly
            if isinstance(data, dict) and "value" in data:
                records = data["value"]
            elif isinstance(data, list):
                records = data
            else:
                records = []
            
            # Convert to DataFrame
            df = pd.DataFrame(records)
            
            # Save as actual CSV
            file_path = os.path.join(data_folder, f"NGER.{dataset_id}.csv")
            df.to_csv(file_path, index=False)
            print(f"{file_path} saved successfully.")
            
        except Exception as e:
            print(f"Failed to parse dataset {dataset_id}: {e}")
    else:
        print(f"Failed to download {dataset_id}, status code: {response.status_code}")

/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0075.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0076.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0077.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0078.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0079.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0080.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0081.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0082.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0083.csv saved successfully.
/Users/ineerajrajeev/Desktop/Assignment 2 DE/data/NGER.ID0243.csv saved successfully.


### What this code does

- **Initializes workspace**: Sets `cwd` to the current working directory, ensures a `data` folder exists.
- **Configures headless browser**: Starts Chrome in headless mode with a tall window to load expandable content.
- **Navigates to target page**: Opens the CER Large-scale Renewable Energy data page and waits for elements.

### Interaction and scraping flow

- **Expands hidden sections**: Finds all “Show” buttons under “About this Table” and clicks each to reveal content, with scroll-into-view and retries per button.
- **Finds relevant links**:
  - Searches for anchors whose `href` starts with `/document`.
  - Filters by link text containing:
    - “Power stations and projects – accredited”
    - “Power stations and projects – committed”
    - “Power stations and projects – probable”
  - Normalizes to absolute URLs and deduplicates.
- **Downloads files**:
  - For each matched link, performs an HTTP GET.
  - On 200 OK, writes the response content to the `data` folder as `<last-path-segment>.csv`.
  - Logs success or failure per file.

### Error handling and cleanup

- **Resilient clicking**: Logs and skips any “Show” button that fails to click; continues processing others.
- **Graceful fallbacks**: Reports when no CSV links are found after expansion.
- **Structured exceptions**: Catches any script-level errors and prints a clear message.
- **Always quits browser**: Ensures `driver.quit()` runs in `finally` to close the headless browser.

In [42]:
# Alternative: Download CER power station files by parsing the page
import requests
from bs4 import BeautifulSoup
import os

data_folder = os.path.join(os.getcwd(), "data")
os.makedirs(data_folder, exist_ok=True)

print("Fetching CER data page...")
url = "https://cer.gov.au/markets/reports-and-data/large-scale-renewable-energy-data"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all links that might be CSV downloads
    target_texts = [
        "accredited",
        "committed", 
        "probable"
    ]
    
    found_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href', '')
        text = link.get_text(strip=True).lower()
        
        # Look for document links containing our target words
        if '/document' in href or '.csv' in href:
            for target in target_texts:
                if target in text or target in href:
                    full_url = href if href.startswith('http') else f"https://cer.gov.au{href}"
                    found_links.append((target, text, full_url))
                    break
    
    print(f"Found {len(found_links)} potential download links:")
    for target, text, link in found_links:
        print(f"  - [{target}] {text[:60]}: {link[:80]}")
    
    # Download the files
    for target, text, link in found_links:
        filename = f"power-stations-and-projects-{target}.csv"
        file_path = os.path.join(data_folder, filename)
        
        try:
            print(f"\nDownloading {filename}...")
            r = requests.get(link, timeout=30)
            if r.status_code == 200:
                with open(file_path, 'wb') as f:
                    f.write(r.content)
                print(f"✓ Saved to {file_path} ({len(r.content)} bytes)")
            else:
                print(f"✗ Failed, status: {r.status_code}")
        except Exception as e:
            print(f"✗ Error: {e}")
else:
    print(f"Failed to fetch page, status: {response.status_code}")

Fetching CER data page...
Found 5 potential download links:
  - [accredited] file typefile type icontotal lgcs and capacity of accredited: https://cer.gov.au/document/total-lgcs-and-capacity-accredited-power-stations-20
  - [accredited] file typefile type icontotal lgcs and capacity of accredited: https://cer.gov.au/document/total-lgcs-and-capacity-accredited-power-stations-20
  - [accredited] file typefile type iconpower stations and projects – accredi: https://cer.gov.au/document/power-stations-and-projects-accredited
  - [committed] file typefile type iconpower stations and projects – committ: https://cer.gov.au/document/power-stations-and-projects-committed
  - [probable] file typefile type iconpower stations and projects – probabl: https://cer.gov.au/document/power-stations-and-projects-probable

✓ Saved to /Users/ineerajrajeev/Desktop/Assignment 2 DE/data/power-stations-and-projects-accredited.csv (12546 bytes)

✓ Saved to /Users/ineerajrajeev/Desktop/Assignment 2 DE/data/power-s

In [43]:
# Set up directories
cwd = os.getcwd()
data_folder = os.path.join(cwd, "data")
os.makedirs(data_folder, exist_ok=True)

# Set up headless browser
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,4000")

driver = webdriver.Chrome(options=options)
driver.get("https://cer.gov.au/markets/reports-and-data/large-scale-renewable-energy-data")
wait = WebDriverWait(driver, 15)

try:
    # Click all "Show" buttons under "About this Table"
    buttons = wait.until(EC.presence_of_all_elements_located((
        By.XPATH, "//button[.//div[text()='About this Table'] and .//span[text()='Show']]"
    )))
    print(f"Found {len(buttons)} 'Show' buttons. Clicking each to reveal content...")

    for idx, button in enumerate(buttons):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", button)
            print(f"Clicked Show button #{idx+1}")
            time.sleep(2)
        except Exception as e:
            print(f"Could not click Show button #{idx+1}: {e}")

    # Look for target CSV links
    link_texts = [
        "Power stations and projects – accredited",
        "Power stations and projects – committed",
        "Power stations and projects – probable"
    ]

    anchors = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/document')]")
    csv_links = []
    seen_urls = set()

    for a in anchors:
        text = a.text.strip()
        href = a.get_attribute("href")
        if any(name in text for name in link_texts):
            full_url = href if href.startswith("http") else "https://cer.gov.au" + href
            if full_url not in seen_urls:
                csv_links.append((text, full_url))
                seen_urls.add(full_url)

    if not csv_links:
        print("No CSV links found after clicking all Show buttons.")
    else:
        print("Found CSV download links:")
        for text, url in csv_links:
            print(f" - {text}: {url}")

        # Download files
        for text, url in csv_links:
            file_name = url.split("/")[-1] + ".csv"
            print(f"Downloading '{text}' from {url}")
            resp = requests.get(url)
            if resp.status_code == 200:
                file_path = os.path.join(data_folder, file_name)
                with open(file_path, "wb") as f:
                    f.write(resp.content)
                print(f"Saved to: {file_path}")
            else:
                print(f"Failed to download {file_name}, status: {resp.status_code}")

except Exception as e:
    print("Script error:", e)

finally:
    driver.quit()

Script error: Message: 



### What this code does

- **Prepares output directory**: Ensures a `data` folder exists under `cwd`.
- **Defines target reports**: Looks specifically for two ABS report titles:
  - “Population and people, ASGS, LGA, and RA, 2011, 2016-2024”
  - “Economy and industry, ASGS and LGA, 2011, 2016-2024”
- **Configures headless browser**: Starts Chrome headless with a tall window to ensure content loads.

### Page navigation and extraction

- **Opens ABS methodology page**: Loads `https://www.abs.gov.au/methodologies/data-region-methodology/2011-24` and waits briefly.
- **Locates downloadable blocks**: Finds elements with class `file-description-link-formatter`.
- **Filters by title**:
  - Reads each block’s `<h4>` title.
  - If the title matches one of the targets, it proceeds to download.

### Downloading files

- **Resolves file URL**: Extracts the `<a>` link in the block; prefixes with `https://www.abs.gov.au` if relative.
- **Saves content**:
  - Performs an HTTP GET.
  - On 200 OK, writes the file to `data/<filename>`.
  - Logs success; otherwise logs the HTTP status.

### Error handling and cleanup

- **No matches**: Prints a message if no targeted files were found.
- **Exceptions**: Catches and logs any runtime errors.
- **Always quits browser**: Closes the headless Chrome in `finally` to free resources.

In [44]:
# Setup download folder
data_folder = os.path.join(cwd, "data")
os.makedirs(data_folder, exist_ok=True)

# Target report titles to look for
target_titles = [
    "Population and people, ASGS, LGA, and RA, 2011, 2016-2024",
    "Economy and industry, ASGS and LGA, 2011, 2016-2024"
]

# Setup headless browser
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,3000")

driver = webdriver.Chrome(options=options)
driver.get("https://www.abs.gov.au/methodologies/data-region-methodology/2011-24")
time.sleep(5)

try:
    # Find each downloadable file block on the page
    blocks = driver.find_elements(By.CLASS_NAME, "file-description-link-formatter")

    found = 0
    for block in blocks:
        title_elem = block.find_element(By.TAG_NAME, "h4")
        title_text = title_elem.text.strip()

        if title_text in target_titles:
            print(f"Found: {title_text}")

            download_link = block.find_element(By.TAG_NAME, "a")
            file_url = download_link.get_attribute("href")
            if file_url.startswith("/"):
                file_url = "https://www.abs.gov.au" + file_url

            file_name = file_url.split("/")[-1]
            file_path = os.path.join(data_folder, file_name)

            print(f"Downloading from {file_url}")
            r = requests.get(file_url)
            if r.status_code == 200:
                with open(file_path, "wb") as f:
                    f.write(r.content)
                print(f"Saved to {file_path}")
                found += 1
            else:
                print(f"Failed to download {file_url} (status {r.status_code})")

    if found == 0:
        print("No matching files found.")

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    driver.quit()


Found: Population and people, ASGS, LGA, and RA, 2011, 2016-2024


KeyboardInterrupt: 

# Data processing and ingestion

## Imports

In [45]:
import pandas as pd
import numpy as np
import duckdb
import uuid
import os
import requests
from urllib.parse import quote
import time

In [46]:
# Set pandas option to avoid FutureWarning for replace
pd.set_option('future.no_silent_downcasting', True)

### What this function does

- **Loads and cleans NGER CSVs** from the `data` folder and returns a consolidated, cleaned `DataFrame` ready for analysis.

### Processing steps

- **Validate inputs**:
  - Ensures `data/` exists and contains `NGER*.csv` files; otherwise returns an empty `DataFrame`.
- **Read and harmonize columns per file**:
  - Reads each `NGER*.csv`.
  - Detects a state column case-insensitively (any column containing “state”) and renames it to `State`.
  - Keeps only available columns from:
    - `Reporting entity`, `Facility name`, `State`,
    - `Electricity production GJ`, `Total scope 1 emissions t CO2 e`,
    - `Total scope 2 emissions t CO2 e`, `Emission intensity t CO2 e MWh`.
  - Skips files without a detectable `State` column.
- **Combine datasets**:
  - Concatenates all per-file subsets into a single `DataFrame`.

### Data cleaning rules

- **Row filtering**:
  - Drops rows with missing `State` or `Emission intensity t CO2 e MWh` (if both columns exist).
- **Numeric coercion and imputation**:
  - For each of the following columns (if present):
    - `Electricity production GJ`
    - `Total scope 1 emissions t CO2 e`
    - `Total scope 2 emissions t CO2 e`
    - `Emission intensity t CO2 e MWh`
  - Converts to numeric (`errors='coerce'`), then fills missing values with the mean for that `State`.

### Error handling and output

- **Resilience**: Logs and skips unreadable/problematic files; continues processing the rest.
- **Graceful fallbacks**: Returns an empty `DataFrame` if no valid files were processed or on unexpected errors.
- **Success path**: Prints progress messages and returns a copy of the cleaned, stacked `DataFrame`.

In [47]:
def load_and_clean_nger_data():
    print("Processing NGER data...")
    try:
        data_files_path = 'data'
        if not os.path.exists(data_files_path):
            print(f"Warning: '{data_files_path}' directory not found. Skipping NGER data processing.")
            return pd.DataFrame()

        data_files = [f for f in os.listdir(data_files_path) if f.startswith('NGER') and f.endswith('.csv')]
        if not data_files:
            print(f"Warning: No NGER CSV files found in '{data_files_path}' directory. Skipping NGER data processing.")
            return pd.DataFrame()

        required_cols = [
            'Reporting entity', 'Facility name', 'State',
            'Electricity production GJ', 'Total scope 1 emissions t CO2 e',
            'Total scope 2 emissions t CO2 e', 'Emission intensity t CO2 e MWh'
        ]

        all_dfs = []
        for file_name in data_files:
            file_path = os.path.join(data_files_path, file_name)
            try:
                df_temp = pd.read_csv(file_path, header=0)
                state_col_name = None
                for col in df_temp.columns:
                    if 'state' in str(col).lower():
                        state_col_name = col
                        break
                if state_col_name:
                    df_temp.rename(columns={state_col_name: 'State'}, inplace=True)

                    cols_to_keep = [col for col in required_cols if col in df_temp.columns]
                    all_dfs.append(df_temp[cols_to_keep])
                else:
                    print(f"Warning: No 'State' column found in {file_name}. Skipping.")

            except Exception as e:
                print(f"Warning: Could not read or process file {file_name}. Error: {e}. Skipping.")

        if not all_dfs:
            print("Warning: No valid NGER files could be processed.")
            return pd.DataFrame()

        stacked_df = pd.concat(all_dfs, ignore_index=True)

        if 'State' in stacked_df.columns and 'Emission intensity t CO2 e MWh' in stacked_df.columns:
            stacked_df = stacked_df.dropna(subset=['State', 'Emission intensity t CO2 e MWh'])

        cols_to_impute = [
            'Electricity production GJ', 'Total scope 1 emissions t CO2 e',
            'Total scope 2 emissions t CO2 e', 'Emission intensity t CO2 e MWh'
        ]
        for col in cols_to_impute:
            if col in stacked_df.columns:
                stacked_df[col] = pd.to_numeric(stacked_df[col], errors='coerce')
                stacked_df[col] = stacked_df.groupby('State')[col].transform(lambda x: x.fillna(x.mean()))

        print("NGER data processed successfully.")
        return stacked_df.copy()
    except Exception as e:
        print(f"An error occurred during NGER data processing: {e}")
        return pd.DataFrame()


### What this function does

- **Geocodes a place name in Australia** using OpenStreetMap Nominatim, returning `(latitude, longitude)` or `(None, None)` if not found.

### How it works

- **Builds query fallbacks**:
  - Full: `"<name> <state> Australia"`
  - Shortened: `"<first-word-of-name> <state> Australia"`
  - Optional: `"Australia postcode <postcode>"` when provided and non-null.
- **Polite request behavior**:
  - Sleeps 1 second before each request to respect Nominatim usage policies.
  - Sends a custom `User-Agent`.
- **Requests and parsing**:
  - Calls Nominatim search API with `format=json`, `limit=1`, `countrycodes=au`.
  - On 200 OK with at least one result, returns the first result’s `lat`, `lon` as floats.
  - Catches exceptions and silently tries the next fallback.

### Return value

- **Success**: `(lat: float, lon: float)`
- **Failure after all fallbacks**: `(None, None)`

In [ ]:
def geocode_with_fallbacks(name, state, postcode=None):
    headers = {'User-Agent': 'power-stations-project/1.0 (neerajshetkar@gmail.com)'}
    queries = [f"{name} {state} Australia", f"{name.split(' ')[0]} {state} Australia"]
    if postcode and pd.notna(postcode):
        queries.append(f"Australia postcode {postcode}")
    for query in queries:
        time.sleep(1)
        url = f"https://nominatim.openstreetmap.org/search?q={quote(query)}&format=json&limit=1&countrycodes=au"
        try:
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200 and response.json():
                result = response.json()[0]
                return float(result['lat']), float(result['lon'])
        except Exception:
            continue
    return None, None

### What this function does

- **Geocodes a DataFrame of locations** by adding `lat` and `lon` columns using `geocode_with_fallbacks`, and reports how many rows were successfully geocoded.

### How it works

- **Initializes output columns**: Sets `df['lat']` and `df['lon']` to `None`.
- **Iterates over rows**:
  - Calls `geocode_with_fallbacks` with `Name_clean`, `State`, and optional `Postcode`.
  - If coordinates are returned, writes them into the row and increments a success counter.
- **Reports and returns**:
  - Prints a summary like `Geocoding complete: X/N successful.`
  - Returns the updated `df` and the `successful_geocodes` count.

In [49]:
def geocode_data(df):
    df['lat'] = None
    df['lon'] = None
    successful_geocodes = 0
    for i, row in df.iterrows():
        lat, lon = geocode_with_fallbacks(row['Name_clean'], row['State'], row.get('Postcode'))
        if lat and lon:
            df.at[i, 'lat'] = lat
            df.at[i, 'lon'] = lon
            successful_geocodes += 1
    print(f"Geocoding complete: {successful_geocodes}/{len(df)} successful.")
    return df, successful_geocodes

### What this function does

- **Backfills missing coordinates** by imputing `lat` and `lon` for rows with null `lat` using the state-level mean coordinates.

### How it works

- **Computes state means**: Groups by `State` and calculates mean `lat`/`lon`.
- **Fills nulls**: For each row where `lat` is null, replaces `lat` and `lon` with the corresponding state mean if available.
- **Returns**: The updated `DataFrame` with fewer missing coordinates.

### Notes

- Assumes `lat`/`lon` are numeric or coercible to numeric.
- Rows in states with no computed mean remain unchanged.

In [50]:
def fill_null_coords(df):
    state_coords = df.groupby('State')[['lat', 'lon']].mean()
    for i, row in df[df['lat'].isnull()].iterrows():
        if row['State'] in state_coords.index:
            df.at[i, 'lat'] = state_coords.loc[row['State'], 'lat']
            df.at[i, 'lon'] = state_coords.loc[row['State'], 'lon']
    return df

### What this function does

- **Loads, standardizes, and enriches CER power station datasets**, then geocodes locations and imputes missing coordinates. Returns a cleaned, consolidated `DataFrame`.

### Processing steps

- **Load source files**: Reads three CSVs from `data/`:
  - `power-stations-and-projects-probable.csv`
  - `power-stations-and-projects-committed.csv`
  - `power-stations-and-projects-accredited.csv`
- **Normalize schema**:
  - Renames columns to a common set (`Name`, `State`, `Capacity`, `Fuel`, plus date fields).
  - Adds `Status` column with values: `Probable`, `Committed`, `Accredited`.
  - Creates `Name_clean` as a trimmed, title-cased version of `Name`.
  - Parses date columns (`Committed_Date`, `Accreditation_Start`, `Approval_Date`) with `dd/mm/YYYY`; coerces invalid to `NaT`.
  - Normalizes `Postcode` to string and replaces `'nan'` with `None` when present.
- **Select and clean fields**:
  - Chooses relevant columns per dataset and drops rows with missing `Capacity`.
  - Converts `Capacity` to numeric and trims whitespace from `State`.
- **Combine and filter**:
  - Concatenates all three datasets.
  - Keeps only records with `Capacity > 10` MW.

### Geospatial enrichment

- **Geocode**: Calls `geocode_data(df)` to add `lat`/`lon` using `Name_clean`, `State`, and optional `Postcode`.
- **Impute missing coords**: Calls `fill_null_coords(df)` to fill remaining null coordinates with state-level mean latitude/longitude.

### Output and errors

- **Returns**: The enriched, geocoded `DataFrame`.
- **Logs**: Prints progress messages at key steps.
- **Error handling**:
  - If files not found: warns and returns empty `DataFrame`.
  - On other exceptions: logs error and returns empty `DataFrame`.

In [52]:
def load_and_clean_cer_data():
    print("Processing CER data...")
    try:
        probable_df = pd.read_csv('data/power-stations-and-projects-probable.csv')
        committed_df = pd.read_csv('data/power-stations-and-projects-committed.csv')
        accredited_df = pd.read_csv('data/power-stations-and-projects-accredited.csv')

        probable_df.rename(columns={'Project Name': 'Name', 'State ': 'State', 'MW Capacity': 'Capacity', 'Fuel Source': 'Fuel'}, inplace=True)
        committed_df.rename(columns={'Project Name': 'Name', 'State ': 'State', 'MW Capacity': 'Capacity', 'Fuel Source': 'Fuel', 'Committed Date (Month/Year)': 'Committed_Date'}, inplace=True)
        accredited_df.rename(columns={'Power station name': 'Name', 'State': 'State', 'Installed capacity (MW)': 'Capacity', 'Fuel Source (s)': 'Fuel', 'Accreditation start date': 'Accreditation_Start', 'Approval date': 'Approval_Date'}, inplace=True)

        probable_df['Status'] = 'Probable'
        committed_df['Status'] = 'Committed'
        accredited_df['Status'] = 'Accredited'

        for df in [probable_df, committed_df, accredited_df]:
            df['Name_clean'] = df['Name'].str.strip().str.title()
            for col in ['Committed_Date', 'Accreditation_Start', 'Approval_Date']:
                if col in df.columns:
                    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')
            if 'Postcode' in df.columns:
                df['Postcode'] = df['Postcode'].astype(str).replace('nan', None)

        probable_df = probable_df[['Name', 'Name_clean', 'State', 'Capacity', 'Fuel', 'Status']].dropna(subset=['Capacity'])
        committed_df = committed_df[['Name', 'Name_clean', 'State', 'Capacity', 'Fuel', 'Status', 'Committed_Date']].dropna(subset=['Capacity'])
        accredited_df = accredited_df[['Name', 'Name_clean', 'State', 'Capacity', 'Fuel', 'Status', 'Postcode', 'Accreditation_Start', 'Approval_Date']].dropna(subset=['Capacity'])

        for df in [probable_df, committed_df, accredited_df]:
            df['Capacity'] = pd.to_numeric(df['Capacity'], errors='coerce')
            df['State'] = df['State'].str.strip()

        df = pd.concat([probable_df, committed_df, accredited_df], ignore_index=True)
        df = df[df['Capacity'] > 10].reset_index(drop=True)

        print("CER data loaded and cleaned. Starting geocoding...")
        df, _ = geocode_data(df)
        df = fill_null_coords(df)
        print("CER data processed successfully.")
        return df
    except FileNotFoundError:
        print("Warning: CER data files not found in 'Dataset' directory. Skipping CER data processing.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred during CER data processing: {e}")
        return pd.DataFrame()

In [ ]:
# Load CER data without geocoding (faster alternative)
print("Loading CER data with approximate state-level coordinates...")
try:
    probable_df = pd.read_csv('data/power-stations-and-projects-probable.csv')
    committed_df = pd.read_csv('data/power-stations-and-projects-committed.csv')
    accredited_df = pd.read_csv('data/power-stations-and-projects-accredited.csv')

    # Standardize columns
    probable_df.rename(columns={'Project Name': 'Name', 'State ': 'State', 'MW Capacity': 'Capacity', 'Fuel Source': 'Fuel'}, inplace=True)
    committed_df.rename(columns={'Project Name': 'Name', 'State ': 'State', 'MW Capacity': 'Capacity', 'Fuel Source': 'Fuel'}, inplace=True)
    accredited_df.rename(columns={'Power station name': 'Name', 'State': 'State', 'Installed capacity (MW)': 'Capacity', 'Fuel Source (s)': 'Fuel'}, inplace=True)

    probable_df['Status'] = 'Probable'
    committed_df['Status'] = 'Committed'
    accredited_df['Status'] = 'Accredited'

    # Combine datasets
    cer_df = pd.concat([probable_df, committed_df, accredited_df], ignore_index=True)
    cer_df['Capacity'] = pd.to_numeric(cer_df['Capacity'], errors='coerce')
    cer_df['State'] = cer_df['State'].str.strip()
    cer_df = cer_df[cer_df['Capacity'] > 10].reset_index(drop=True)
    
    # Add approximate state-level coordinates (capital city locations)
    state_coords = {
        'NSW': (-33.8688, 151.2093),
        'VIC': (-37.8136, 144.9631),
        'QLD': (-27.4698, 153.0251),
        'SA': (-34.9285, 138.6007),
        'WA': (-31.9505, 115.8605),
        'TAS': (-42.8821, 147.3272),
        'NT': (-12.4634, 130.8456),
        'ACT': (-35.2809, 149.1300)
    }
    
    cer_df['lat'] = cer_df['State'].map(lambda s: state_coords.get(s, (None, None))[0])
    cer_df['lon'] = cer_df['State'].map(lambda s: state_coords.get(s, (None, None))[1])
    
    print(f"✓ CER data loaded: {len(cer_df)} power stations")
    print(f"✓ Columns: {list(cer_df.columns)}")
    print(f"✓ Coordinates added: {cer_df['lat'].notna().sum()}/{len(cer_df)} stations")
    
except Exception as e:
    print(f"Error loading CER data: {e}")
    cer_df = pd.DataFrame()

Processing CER data...
CER data loaded and cleaned. Starting geocoding...


KeyboardInterrupt: 

### What this function does

- **Loads and cleans ABS population tables** from an Excel workbook, standardizes their schema, imputes missing numeric values, and returns a combined `DataFrame`.

### Processing steps

- **File check**: Verifies `data/14100DO0001_2011-24.xlsx` exists; otherwise returns an empty `DataFrame`.
- **Read sheets**: Loads “Table 1”, “Table 2”, and “Table 3” with `skiprows=5`.

### Cleaning logic per sheet (`clean_df`)

- **Header normalization**:
  - Uses the first data row as headers, then resets rows.
  - Slugifies headers to lowercase with underscores.
  - Renames first two columns to `label` and `code`.
- **Row filtering**:
  - Keeps rows with non-null `code` and excludes notes/copyright rows.
- **Type handling**:
  - If present, converts `year` to nullable integer.
  - Replaces `"-"` and `"nil"` with `NaN`.
  - Converts non-identifier object columns to numeric (coercing errors to `NaN`).
- **Imputation**:
  - For each numeric column, fills `NaN` with the mean within the same `label`.
- **Final filter**:
  - Drops rows where `year` is missing.

### Output

- **Concatenates** the three cleaned tables into `abs_df` and returns it.
- **Error handling**: On any exception, logs and returns an empty `DataFrame`.

In [ ]:
def load_and_clean_abs_data():
    print("Processing ABS data...")
    try:
        pop_file = "data/14100DO0001_2011-24.xlsx"
        if not os.path.exists(pop_file):
            print("Warning: ABS data file not found. Skipping ABS data processing.")
            return pd.DataFrame()

        pop_table1 = pd.read_excel(pop_file, sheet_name="Table 1", skiprows=5)
        pop_table2 = pd.read_excel(pop_file, sheet_name="Table 2", skiprows=5)
        pop_table3 = pd.read_excel(pop_file, sheet_name="Table 3", skiprows=5)

        def clean_df(df):
            if df.empty: return df
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
            df.columns = [str(col).strip().lower().replace(" ", "_") for col in df.columns]
            df = df.rename(columns={df.columns[0]: "label", df.columns[1]: "code"})
            df = df[df["code"].notna() & ~df["code"].astype(str).str.contains("Note|©", na=False)]
            if 'year' in df.columns:
                df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
            df = df.replace(["-", "nil"], np.nan)
            for col in df.select_dtypes(include=["object"]).columns:
                if col not in ["label", "code"]:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            for col in df.select_dtypes(include="number").columns:
                df[col] = df.groupby("label")[col].transform(lambda x: x.fillna(x.mean()))
            return df.dropna(subset=['year'])  # Changed from dropna() to dropna(subset=['year'])

        pop_table1 = clean_df(pop_table1)
        pop_table2 = clean_df(pop_table2)
        pop_table3 = clean_df(pop_table3)

        abs_df = pd.concat([pop_table1, pop_table2, pop_table3], ignore_index=True)
        print("ABS data processed successfully.")
        return abs_df
    except Exception as e:
        print(f"An error occurred during ABS data processing: {e}")
        return pd.DataFrame()

### What this function does

- **Builds and populates a DuckDB database** (`australian_electricity_data.db`) with standardized tables for states, fuels, power stations, emissions, and ABS demographics, including geospatial points.

### Steps and tables created

- **Init and extensions**
  - Connects to DuckDB, installs/loads `spatial` for geometry support.

- **States**
  - Creates `states(state_id, state_code, state_name)`.
  - Extracts unique state codes from `NGER` and `CER` inputs, maps to full names, and upserts.

- **Fuels**
  - If `CER` has `Fuel`, creates `fuels(fuel_id, fuel_type)` and inserts unique fuels.

- **Power stations**
  - If `CER` is non-empty, creates `power_stations(...)` with:
    - UUID `station_id`, `name`, `capacity_mw`, `status`
    - `latitude`, `longitude`, `geom` as `ST_Point(lon, lat)` when both present
    - Foreign keys to `states(state_id)` and `fuels(fuel_id)`
  - Maps `State` and `Fuel` to their IDs and inserts all rows.

- **Emissions**
  - If `NGER` is non-empty, creates `emissions(...)` with:
    - UUID `emission_id`, `reporting_entity`, `facility_name`,
      `electricity_production_gj`, `scope1_emissions_t_co2e`, `scope2_emissions_t_co2e`
    - `state_id` as FK
  - Maps `State` to `state_id` and inserts.

- **ABS demographics**
  - If `ABS` is non-empty, creates `abs_demographics(...)` with:
    - UUID `demographic_id`, `year`, `metric_name`, `metric_value`, `state_id` (FK)
  - Melts numeric columns into long format, drops nulls, heuristically assigns `state_code` by matching `label` to state names/codes, and bulk inserts.

- **Finalize**
  - Prints success messages and closes the connection.

In [ ]:
def create_and_populate_database(nger_df, cer_df, abs_df):
    print("Creating and populating the database...")
    con = duckdb.connect('australian_electricity_data.db')
    con.execute("INSTALL spatial; LOAD spatial;")
    con.execute("""
    CREATE TABLE IF NOT EXISTS states (
        state_id VARCHAR PRIMARY KEY,
        state_code VARCHAR UNIQUE,
        state_name VARCHAR
    )
    """)
    state_mapping = {'NSW': 'New South Wales', 'VIC': 'Victoria', 'QLD': 'Queensland', 'SA': 'South Australia', 'WA': 'Western Australia', 'TAS': 'Tasmania', 'NT': 'Northern Territory', 'ACT': 'Australian Capital Territory'}
    all_states = set()
    if not nger_df.empty and 'State' in nger_df.columns: all_states.update(nger_df['State'].unique())
    if not cer_df.empty and 'State' in cer_df.columns: all_states.update(cer_df['State'].unique())
    state_data = [(str(uuid.uuid4()), code, state_mapping.get(code)) for code in all_states if code in state_mapping]
    if state_data:
        states_df = pd.DataFrame(state_data, columns=['state_id', 'state_code', 'state_name'])
        con.register('states_view', states_df)
        con.execute("""
            INSERT INTO states (state_id, state_code, state_name)
            SELECT state_id, state_code, state_name FROM states_view
            ON CONFLICT (state_code) DO NOTHING
        """)
        con.unregister('states_view')

    state_ids = dict(con.execute("SELECT state_code, state_id FROM states").fetchall())

    if not cer_df.empty and 'Fuel' in cer_df.columns:
        con.execute("CREATE TABLE IF NOT EXISTS fuels (fuel_id VARCHAR PRIMARY KEY, fuel_type VARCHAR UNIQUE)")

        fuel_data = [(str(uuid.uuid4()), fuel) for fuel in cer_df['Fuel'].unique() if pd.notna(fuel)]
        if fuel_data:
            fuels_df = pd.DataFrame(fuel_data, columns=['fuel_id', 'fuel_type'])
            con.register('fuels_view', fuels_df)
            con.execute("""
                INSERT INTO fuels (fuel_id, fuel_type)
                SELECT fuel_id, fuel_type FROM fuels_view
                ON CONFLICT (fuel_type) DO NOTHING
            """)
            con.unregister('fuels_view')
        fuel_ids = dict(con.execute("SELECT fuel_type, fuel_id FROM fuels").fetchall())
    if not cer_df.empty:
        con.execute("""
        CREATE OR REPLACE TABLE power_stations (
            station_id VARCHAR PRIMARY KEY, name VARCHAR, capacity_mw FLOAT, status VARCHAR,
            latitude FLOAT, longitude FLOAT, geom GEOMETRY,
            state_id VARCHAR REFERENCES states(state_id), fuel_id VARCHAR REFERENCES fuels(fuel_id)
        )
        """)
        ps_df = cer_df.copy()
        ps_df['station_id'] = [str(uuid.uuid4()) for _ in range(len(ps_df))]
        ps_df['state_id'] = ps_df['State'].map(state_ids)
        ps_df['fuel_id'] = ps_df['Fuel'].map(fuel_ids)

        con.register('ps_view', ps_df)
        con.execute("""
            INSERT INTO power_stations
            SELECT
                station_id, Name AS name, Capacity AS capacity_mw, Status AS status,
                lat AS latitude, lon AS longitude,
                CASE WHEN lon IS NOT NULL AND lat IS NOT NULL THEN ST_Point(lon, lat) ELSE NULL END AS geom,
                state_id, fuel_id
            FROM ps_view
        """)
        con.unregister('ps_view')

    if not nger_df.empty:
        con.execute("""
        CREATE OR REPLACE TABLE emissions (
            emission_id VARCHAR PRIMARY KEY, reporting_entity VARCHAR, facility_name VARCHAR,
            electricity_production_gj FLOAT, scope1_emissions_t_co2e FLOAT,
            scope2_emissions_t_co2e FLOAT, state_id VARCHAR REFERENCES states(state_id)
        )
        """)
        emissions_df = nger_df.copy()
        emissions_df['emission_id'] = [str(uuid.uuid4()) for _ in range(len(emissions_df))]
        emissions_df['state_id'] = emissions_df['State'].map(state_ids)

        # Rename columns to match SQL identifiers (no spaces, lowercase, underscores)
        rename_map = {
            "Reporting entity": "reporting_entity",
            "Facility name": "facility_name",
            "Electricity production GJ": "electricity_production_gj",
            "Total scope 1 emissions t CO2 e": "scope1_emissions_t_co2e",
            "Total scope 2 emissions t CO2 e": "scope2_emissions_t_co2e"
        }
        # Only rename if columns exist
        for old, new in rename_map.items():
            if old in emissions_df.columns:
                emissions_df.rename(columns={old: new}, inplace=True)

        # Ensure all required columns exist
        required_cols = [
            "emission_id", "reporting_entity", "facility_name",
            "electricity_production_gj", "scope1_emissions_t_co2e",
            "scope2_emissions_t_co2e", "state_id"
        ]
        missing_cols = [col for col in required_cols if col not in emissions_df.columns]
        if missing_cols:
            print(f"Warning: The following columns are missing in NGER data and will be filled with None: {missing_cols}")
            for col in missing_cols:
                emissions_df[col] = None

        con.register('emissions_view', emissions_df)
        con.execute("""
            INSERT INTO emissions
            SELECT
                emission_id, reporting_entity, facility_name, electricity_production_gj,
                scope1_emissions_t_co2e, scope2_emissions_t_co2e, state_id
            FROM emissions_view
        """)
        con.unregister('emissions_view')

    if not abs_df.empty:
        con.execute("""
        CREATE OR REPLACE TABLE abs_demographics (
            demographic_id VARCHAR PRIMARY KEY, year INTEGER, metric_name VARCHAR,
            metric_value FLOAT, state_id VARCHAR REFERENCES states(state_id)
        )
        """)
        numeric_cols = [col for col in abs_df.columns if col not in ['label', 'code', 'year'] and abs_df[col].dtype in ['float64', 'int64', 'Int64']]
        if numeric_cols:
            abs_melted = abs_df.melt(id_vars=['label', 'code', 'year'], value_vars=numeric_cols, var_name='metric_name', value_name='metric_value')
            abs_melted = abs_melted.dropna(subset=['metric_value'])  # Remove rows with null values
            
            abs_data = []
            for _, row in abs_melted.iterrows():
                state_code = None
                label_lower = str(row['label']).lower()
                for code, name in state_mapping.items():
                    if name.lower() in label_lower or code.lower() in label_lower:
                        state_code = code
                        break
                
                if state_code and state_ids.get(state_code) and pd.notna(row.get('metric_value')):
                    abs_data.append((
                        str(uuid.uuid4()), 
                        int(row.get('year')) if pd.notna(row.get('year')) else None, 
                        str(row.get('metric_name')), 
                        float(row.get('metric_value')), 
                        state_ids.get(state_code)
                    ))
            
            if abs_data: 
                con.executemany("INSERT INTO abs_demographics (demographic_id, year, metric_name, metric_value, state_id) VALUES (?, ?, ?, ?, ?)", abs_data)
                print(f"Inserted {len(abs_data)} ABS demographic records.")

    print("Database created and populated successfully.")
    con.close()

### What this code does

- **Verifies the database**:
  - Checks that `australian_electricity_data.db` exists.
  - Connects to DuckDB, lists all tables, and prints the first 5 rows from each.
  - Catches and reports read errors per table, then closes the connection.

### Script entrypoint workflow

- **Runs data pipelines**:
  - Calls `load_and_clean_nger_data()`, `load_and_clean_cer_data()`, and `load_and_clean_abs_data()`.
- **Conditional database build**:
  - If all three are empty, prints a halt message.
  - Otherwise, calls `create_and_populate_database(...)`, then `verify_database()` to preview contents, and prints a completion message.

In [ ]:
def verify_database():
    db_file = 'australian_electricity_data.db'
    if not os.path.exists(db_file):
        print(f"Database file '{db_file}' not found. Cannot verify.")
        return
    print("\nVerifying database contents...")
    con = duckdb.connect(db_file)
    tables = con.execute("SHOW TABLES").fetchdf()['name'].tolist()
    for table in tables:
        print(f"\n--- Contents of {table} table (first 5 rows) ---")
        try:
            print(con.execute(f"SELECT * FROM {table} LIMIT 5").fetchdf())
        except Exception as e:
            print(f"Could not read from table {table}. Error: {e}")
    con.close()

if __name__ == "__main__":
    nger_df = load_and_clean_nger_data()
    cer_df = load_and_clean_cer_data()
    abs_df = load_and_clean_abs_data()

    if nger_df.empty and cer_df.empty and abs_df.empty:
        print("\nNo data was processed. Halting database creation.")
    else:
        create_and_populate_database(nger_df, cer_df, abs_df)
        verify_database()
        print("\nProcess complete! Database 'australian_electricity_data.db' created.")

Processing NGER data...
NGER data processed successfully.
Processing CER data...
CER data loaded and cleaned. Starting geocoding...


KeyboardInterrupt: 

### What this function does

- **Explores CER data and visualizes capacity by fuel type**. Saves a bar chart and prints a capacity summary.

### Steps

- **Guard**: Skips if `cer_df` is empty.
- **Aggregate**: Groups by `Fuel` and sums `Capacity`, sorted descending.
- **Report**: Prints the total capacity per fuel source.
- **Visualize**:
  - Creates a bar chart using a Viridis palette.
  - Titles and labels axes; rotates x-axis labels; applies `seaborn`-style grid.
  - Saves to `capacity_by_fuel_type.png` and displays the plot.
- **Finish**: Prints completion message.

## Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def explore_and_visualize_data(cer_df):
    """
    Performs data exploration on the cleaned CER data and generates a visualization.
    """
    if cer_df.empty:
        print("\nData Exploration: CER DataFrame is empty. Skipping visualization.")
        return

    print("\n--- Starting Data Exploration ---")

    fuel_capacity = cer_df.groupby('Fuel')['Capacity'].sum().sort_values(ascending=False)

    print("\nKey Finding: Total Power Generation Capacity (MW) by Fuel Source:")
    print(fuel_capacity)

    # --- Data Visualization: Bar Chart of Capacity by Fuel Type ---
    print("\nGenerating visualization...")
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 8))

    fuel_capacity.plot(kind='bar', ax=ax, color=sns.color_palette("viridis", len(fuel_capacity)))

    ax.set_title('Total Power Generation Capacity (MW) by Fuel Type in Australia', fontsize=16, weight='bold')
    ax.set_xlabel('Fuel Type', fontsize=12)
    ax.set_ylabel('Total Capacity (MW)', fontsize=12)
    ax.tick_params(axis='x', rotation=45, labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    plt.tight_layout()

    # Save the figure to a file
    plt.savefig('capacity_by_fuel_type.png')
    print("Visualization saved as 'capacity_by_fuel_type.png'")

    # Display the plot
    plt.show()

explore_and_visualize_data(cer_df)


Data Exploration: CER DataFrame is empty. Skipping visualization.


In [ ]:
def explore_and_visualize_map(cer_df):
    if cer_df.empty or 'lat' not in cer_df.columns or 'lon' not in cer_df.columns:
        print("\nData Exploration: CER DataFrame is empty or missing coordinate columns. Skipping map visualization.")
        return

    print("\n--- Starting Data Exploration for Map Visualization ---")

    df_map = cer_df.dropna(subset=['lat', 'lon', 'Fuel']).copy()

    # --- Data Visualization: Interactive Map ---
    print("Generating interactive map...")

    # Create a color palette for the different fuel types
    unique_fuels = sorted(df_map['Fuel'].unique())
    colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]
    color_map = {fuel: colors[i % len(colors)] for i, fuel in enumerate(unique_fuels)}
    df_map['color'] = df_map['Fuel'].map(color_map)

    # Initialize map centered on Australia
    aus_map = folium.Map(location=[-25.2744, 133.7751], zoom_start=4)

    # Use MarkerCluster for better performance with many points
    marker_cluster = MarkerCluster().add_to(aus_map)

    # Add points to the map
    for idx, row in df_map.iterrows():
        popup_html = f"""
        <b>Name:</b> {row['Name']}<br>
        <b>Capacity (MW):</b> {row['Capacity']}<br>
        <b>Fuel:</b> {row['Fuel']}<br>
        <b>Status:</b> {row['Status']}
        """
        iframe = folium.IFrame(popup_html, width=250, height=100)
        popup = folium.Popup(iframe, max_width=250)

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            popup=popup,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            fill_opacity=0.7
        ).add_to(marker_cluster)

    # --- Add a Legend ---
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 180px; height: auto;
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white; padding: 10px;">
    <b>Fuel Types</b><br>
    '''
    for fuel, color in color_map.items():
        legend_html += f'<i class="fa fa-circle" style="color:{color}"></i> {fuel}<br>'
    legend_html += '</div>'
    aus_map.get_root().html.add_child(folium.Element(legend_html))

    # Display the map inline in the notebook
    from IPython.display import HTML
    map_html = aus_map._repr_html_()
    display(HTML(map_html))
    print("\n--- Map Visualization Complete ---")

explore_and_visualize_map(cer_df)


Data Exploration: CER DataFrame is empty or missing coordinate columns. Skipping map visualization.


### AI usage acknowledgement

This project used AI assistance from ChatGPT (GPT‑5) to help diagnose and fix code issues, improve code quality, and draft concise documentation of functions and workflows. All outputs were reviewed, tested, and integrated by the group.